<a href="https://colab.research.google.com/github/Nhat-Thanh/CS114.L21/blob/main/Final%20Project/SSD%20resnet50/facemask_detection_using_ssd_resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi 

Tue Aug 10 11:37:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#**1. Kết nối với google drive (không cần thiết)**

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

#**2. Tải mô hình đã được huấn luyện trước**

In [ ]:
!gdown --id 112-eCAFJYif-KZNXJh4-u6j6FxUEwWlv
!mkdir training
!unzip training.zip -d training/
!rm training.zip

#**3. Chuẩn bị môi trường để chạy model**

##**3.1. Setup conda enviroment**

###**3.1.1 Tải và cài đặt conda**

In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

###**3.1.2. Cập nhật các gói trong môi trường conda**

In [ ]:
!rm Miniconda3-4.5.4-Linux-x86_64.sh
!conda install --channel defaults conda python=3.9.5 --yes
!conda update --channel defaults --all --yes
!conda update --channel conda-forge --all --yes
!conda install ipykernel --yes
!conda install -c fastchan numpy=1.19.5 --yes
!pip install Cython

In [ ]:
!python --version

Python 3.9.6


##**3.2. Chuẩn bị các công cụ cần thiết**

###**3.2.1. Protoc**

In [ ]:
# Tải gói protoc
!wget https://github.com/protocolbuffers/protobuf/releases/download/v3.17.3/protoc-3.17.3-linux-x86_64.zip

In [ ]:
# Giải nén gói protoc vừa tải
!unzip protoc-3.17.3-linux-x86_64.zip -d protoc

###**3.2.2. Pre-trainded-model từ tensorflow**

In [ ]:
!git clone https://github.com/tensorflow/models.git

###**3.2.3. Protobuf Installation/Compilation**

In [ ]:
%cd models/research/
!/content/protoc/bin/protoc object_detection/protos/*.proto --python_out=.

##**3.3. Cài đặt COCO API**

In [ ]:
%cd /content/

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git
!make --directory=cocoapi/PythonAPI/
!cp -r cocoapi/PythonAPI/pycocotools models/research/

##**3.4. Cài đặt Object Detection API**

In [ ]:
!conda install -c fastchan python=3.8.5 --yes
!pip install tensorflow==2.3.0 tensorflow-gpu==2.3.0

In [ ]:
!cp /content/models/research/object_detection/packages/tf2/setup.py /content/models/research/.
!python -m pip install --use-feature=2020-resolver models/research/.

In [ ]:
!conda install -c fastchan python=3.8.5 --yes
!pip install tensorflow==2.3.0 tensorflow-gpu==2.3.0

#**4. Kiểm tra GPU**

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
print("Num GPUs:", len(physical_devices))

#**5. Detect**

In [ ]:
%cd /content/training

In [ ]:
%%writefile detect_image.py
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import os
import cv2
import time
import numpy as np
from PIL import Image

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

PATH_TO_LABELS = "annotations/label_map.pbtxt"
PATH_TO_SAVED_MODEL = "exported-models/saved_model"
PATH_TO_IMAGES = "image_test/"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
model_predict = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))


category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

image_list = os.listdir(PATH_TO_IMAGES)

start = time.time()
time_avg = 0

# DETECT IMAGES
'''
for image_name in image_list:
    start_time = time.time()
    print(image_name)
    img = cv2.imread(f"image_test/{image_name}")
    image_np = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)

    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = model_predict(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_with_detections = img

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.50,
          agnostic_mode=False)
        
    cv2.imwrite(f"result/{image_name}", image_with_detections)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print('Done! Took {} seconds'.format(elapsed_time))
    time_avg += elapsed_time
    
print(f"time avg: {time_avg / len(limage_list)}")
'''

# DETECT VIDEO

# pass video's name into this funcion
video = cv2.VideoCapture('video2.mp4')
res=(640, 360) #resulotion
fourcc = cv2.VideoWriter_fourcc(*'MP4V') #codec
out = cv2.VideoWriter('output.mp4', fourcc, 25, res)
num_frame = 0

while (video.isOpened()):
    start_time = time.time()
    ret, frame = video.read()
    if ret == False:
      break
    image_np = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)

    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = model_predict(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_with_detections = frame
    # image_with_detections = img

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.5,
          agnostic_mode=False)
    
    out.write(image_with_detections)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print('Done! Took {} seconds'.format(elapsed_time))
    time_avg += elapsed_time
    num_frame += 1

# average time for processing a frame
print(f"time avg: {time_avg / num_frame}")
# render video
out.release()

# elapsed time
end = time.time()
elapsed = end - start
print(f'Done! Took {elapsed} seconds')

Overwriting detect_image.py


In [ ]:
!mkdir -p image_test
!mkdir -p result

In [ ]:
!curl -s http://transfer.sh/1ZnAr2g/video.mp4 -o video.mp4
!curl -s http://transfer.sh/1ZGF1kV/video1.mp4 -o video1.mp4
!curl -s http://transfer.sh/1hlEN6k/video.mp4 -o video2.mp4

In [ ]:
!python detect_image.py

# UNCOMMENT THESE LINES TO SHOW ALL IMAGES
# IF DETECTING VIDEO, COMMENT ALL THESE LINES
# PASS DESIRE IMAGES TO "image_test" FOLDER TO DETECT
# ALL RESULTS ARE SAVED AT "result" FOLDER
'''
from google.colab.patches import cv2_imshow
import cv2
import os
 
image_list = os.listdir("result")
for img_name in image_list:
  image = cv2.imread(f"result/{img_name}")
  print("\n", img_name)
  image = cv2.resize(image, (600, 500))
  cv2_imshow(image)
'''

2021-08-10 11:49:47.770165: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-08-10 11:49:50.446220: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-08-10 11:49:50.452990: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-10 11:49:50.453550: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-08-10 11:49:50.453586: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-08-10 11:49:50.682307: I tensorflow/stream_executor/platform/default

'\nfrom google.colab.patches import cv2_imshow\nimport cv2\nimport os\n \nimage_list = os.listdir("result")\nfor img_name in image_list:\n  image = cv2.imread(f"result/{img_name}")\n  print("\n", img_name)\n  image = cv2.resize(image, (600, 500))\n  cv2_imshow(image)\n'